In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import networkx as nx
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

In [60]:
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."

parser = spacy.load('en')

gatsby = parser(gatsby)

sentences=[]
for span in gatsby.sents:
    sent = ''.join(gatsby[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)

counter = TfidfVectorizer(lowercase=False, 
                          stop_words=None,
                          ngram_range=(1, 1), 
                          analyzer=u'word', 
                          max_df=.5, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

data_counts=counter.fit_transform(sentences)

In [62]:
similarity = data_counts * data_counts.T
# Identifying the sentence with the highest rank.
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks=nx.pagerank(nx_graph, alpha=.85, tol=.00000001)

ranked = sorted(((ranks[i],s) for i,s in enumerate(sentences)),
                reverse=True)
print(ranked[0])


(0.0639701861589625, 'The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men.')


In [65]:
gatsby

In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. "Whenever you feel like criticizing any one," he told me, "just remember that all the people in this world haven't had the advantages that you've had." He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intim

In [82]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
bigrams = []
for word in range(len(gatsby)- 1):
    if ((gatsby[word].is_stop==False) and (gatsby[word].pos_=='NOUN' or gatsby[word].pos_=='ADJ')) or ((gatsby[word + 1].is_stop==False) and (gatsby[word + 1].pos_=='NOUN' or gatsby[word + 1].pos_=='ADJ')):
        bigrams.append(str(gatsby[word]) + ' ' + str(gatsby[word + 1]))
words=set(bigrams)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(bigrams):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in bigrams]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(bigrams)-(len(bigrams)-(i+6)))
        # The potential neighbors.
        nextwords=bigrams[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in bigrams for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

done!


In [83]:
adjacency

,hope .,a politician,hard rock,his dreams,unbroken series,foul dust,a great,great deal,privy to,certain point,...,the end,abortive sorrows,"preoccupation ,",a sense,more vulnerable,the wet,the advantages,gorgeous about,deal more,unusually communicative
hope .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a politician,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
hard rock,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
his dreams,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
unbroken series,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
foul dust,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a great,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
great deal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
privy to,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
certain point,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])


[(0.00849746470947563, 'my father'), (0.00645705818499071, 'men .'), (0.004657230324663844, 'abortive sorrows'), (0.0046558391245408075, 'father gave'), (0.004640659204971256, 'sorrows and')]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [85]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
triigrams = []
for word in range(len(gatsby)- 2):
    if ((gatsby[word].is_stop==False) and (gatsby[word].pos_=='NOUN' or gatsby[word].pos_=='ADJ')) or ((gatsby[word + 1].is_stop==False) and (gatsby[word + 1].pos_=='NOUN' or gatsby[word + 1].pos_=='ADJ')) or ((gatsby[word + 2].is_stop==False) and (gatsby[word + 2].pos_=='NOUN' or gatsby[word + 2].pos_=='ADJ')):
        trigrams.append(str(gatsby[word]) + ' ' + str(gatsby[word + 1]) + ' ' + str(gatsby[word + 2]))
words=set(trigrams)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(trigrams):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in trigrams]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(trigrams)-(len(trigrams)-(i+6)))
        # The potential neighbors.
        nextwords=trigrams[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in trigrams for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

NameError: name 'trigrams' is not defined